In [ ]:
!pip install strands-agents[mistral] python-dotenv

In [11]:
import os
import sys
import json

from pathlib import Path

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    make_submission,
    get_job_paths,
    sanity_check,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    send_message_to_chat
)

from src.models.job_info import JobInfo

In [12]:
DATA_JOBS_DIR = Path('../data_jobs')
DATA_TRAININGS_DIR = Path('../data_trainings')
DATA_MATCH_JOBS_TRAININGS_DIR = Path('../data_match_jobs_trainings')
SUBMISSION_DIR = Path('../submissions')
DATA_SUBMISSION_FEEDBACK_DIR = Path('../data_submission_feedback')

In [13]:
job_data_version = "v3"

In [14]:
jobs_dir = Path('../data/jobs')
trainings_dir = Path('../data/trainings')

In [15]:
def request_feedback(persona_id, result):
    agent_message = "Hello! I'm coming back to you to give you feed back regarding our last interview.\n"

    api_called = False

    if result['predicted_type'] == 'awareness':
        if result['predicted_type'] == 'too_young':
            agent_message += "As your age is below 16, I will only provide you awereness about topics you are interested in.\n"
            agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
        else:
            agent_message += "As your are only interested by awereness (no training nor jib), let's discuss about .\n"
            agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
        resp = send_message_to_chat(agent_message, persona_id)
    elif result['predicted_type'] == 'trainings_only':
        agent_message += "I have identified that you are interested in training only. Your are not seeking for jobs.\n"
        agent_message += "Hereafter if the list of trainings I find for you.\n"
        agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
        agent_message += "\nTRAININGS DESCRIPTION :"

        trainings = result['trainings']
        for training_id in trainings:
            filename = f"{training_id}.md"
            path = trainings_dir / filename
            p = Path(path)
            content = p.read_text(encoding='utf-8', errors='ignore')
            agent_message += "\n\n------ TRAINING -------"
            agent_message += content
        resp = send_message_to_chat(agent_message, persona_id)
        api_called = True
    elif result['predicted_type'] == 'jobs+trainings':
        agent_message += "I have identified that you are interested for jobs opportunities.\n"
        agent_message += "Hereafter if the list of jobs I find for you with associated trainings I recommend you to take.\n"
        agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
        agent_message += "\nTRAININGS DESCRIPTION :\n"

        jobs = result['jobs']

        for job in jobs:
            job_id = job.get("job_id")
            trainings = job.get("suggested_trainings", [])
            filename = f"{job_id}.md"
            path = jobs_dir / filename
            p = Path(path)
            content = p.read_text(encoding='utf-8', errors='ignore')
            agent_message += f"\n\n###Job {job_id}"
            agent_message += "\n##Job Description"
            agent_message += content
            agent_message += f"\n##Suggested Training for Job {job_id}"
            if len(trainings) > 0:
                for training_id in trainings:
                    filename = f"{training_id}.md"
                    path = trainings_dir / filename
                    p = Path(path)
                    content = p.read_text(encoding='utf-8', errors='ignore')
                    agent_message += "\n#Training description"
                    agent_message += content
            else:
                agent_message += "No training suggested for this job"

            resp = send_message_to_chat(agent_message, persona_id)
            api_called = True

    feedback = "ASSITANT:\n" + agent_message + "\n\nUSER:\n" + resp[0]
    return feedback, resp[1], api_called


In [16]:
def hash_dict(data: dict) -> str:
    # Serialize with sorted keys to ensure consistent ordering
    serialized = json.dumps(data, sort_keys=True)
    return hash(serialized)

In [17]:
def get_feedback(persona_id, selected_file, result):
    result_hash = str(hash_dict(result))

    api_called = False

    feedback_save_path = DATA_SUBMISSION_FEEDBACK_DIR / "feedback.json"
    if not feedback_save_path.exists():
        save_json(feedback_save_path, {})
    feedback_db = read_json(feedback_save_path)

    if persona_id not in feedback_db:
        feedback_db[persona_id] = {}
        feedback_persona = {}
    else:
        feedback_persona = feedback_db[persona_id]

    if result_hash in feedback_persona:
        return feedback_persona[result_hash], api_called
    else:
        analysis_result, conversation_id, api_called = request_feedback(persona_id, result)
        feedback_entry = {
            result_hash: {
                "file": selected_file,
                "analysis_result": analysis_result,
                "conversation_id": conversation_id
            }
        }
        feedback_db[persona_id].update(feedback_entry)
        save_json(feedback_save_path, feedback_db)
        return feedback_entry[result_hash], api_called

In [ ]:
selected_file = "results_2025_09_26__19_44_46.json"
results_save_path = SUBMISSION_DIR / selected_file
results = read_json(results_save_path)

api_calls = 0
for result in results:
    #print(result)
    persona_id = result['persona_id']
    feedback_entry, api_called = get_feedback(persona_id, selected_file, result)
    #print(feedback_entry)
    user_response = feedback_entry['analysis_result'].split("USER:")[1].strip()
    print("\n" + "="*50)
    print(persona_id)
    print(user_response)
    
    if api_called is True:
        api_calls = api_calls + 1

print("\n" + "="*50)
print(f"Nb API calls : {api_calls}")


persona_001
oh hi. umm... i think u got it wrong? i want trainig for food safe and how make food good. not just awereness. i wanna lern the mashines and rules so one day i can work. i am 21 but still in school so i need start small. can u help?

persona_002
The Intermediate Food Waste Management Training looks like the best fit for me. It’s in Portuguese, online, and focuses on practical ways to reduce waste, which is exactly what I need.

Does this training include hands-on exercises for measuring waste in production lines? That’s something I really want to learn.

persona_003
Oh, wow, this list is incredibly exciting! I can already feel my passion for sustainability and efficient food production bubbling up just reading through these opportunities. You’ve really captured the essence of what drives me—especially when I see terms like "waste reduction strategies" and "recipe scaling." Those are the kinds of challenges that make me jump out of bed in the morning, ready to tackle the da

FileNotFoundError: [Errno 2] No such file or directory: '../data/jobs/job_010.md'

In [32]:
selected_file = "results_2025_09_26__14_41_18.json"
results_save_path = SUBMISSION_DIR / selected_file
results = read_json(results_save_path)

persona_index = 0
result = results[persona_index]
persona_id = result['persona_id']
feedback_entry, api_called = get_feedback(persona_id, selected_file, result)

In [33]:
print(api_called)

False


In [22]:
selected_file = "results_2025_09_26__14_41_18.json"
results_save_path = SUBMISSION_DIR / selected_file
results = read_json(results_save_path)

persona_index = 0
persona_id = result['persona_id']

result = results[persona_index]

print(result)

result_hash = hash_dict(result)

print(result_hash)

feedback_save_path = DATA_SUBMISSION_HASH_DIR / "feedback.json"
if not feedback_save_path.exists():
    save_json(feedback_save_path, {})
feedback_db = read_json(feedback_save_path)

if persona_id not in feedback_db:
    feedback_db[persona_id] = {}
    
feedback_entry = {
    result_hash: {
        "file": "results_2025_09_26__14_41_18",
        "analysis_result": "analysis_result"
    }
}
feedback_db[persona_id].update(feedback_entry)

result_hash = 123
feedback_entry = {
    result_hash: {
        "session_id": "new",
        "analysis_result": "analysis_result"
    }
}
feedback_db[persona_id].update(feedback_entry)

#if persona_id not in feedback_db:
#    feedback_db[persona_id] = {}

#feedback_entry = {
#    result_hash: {
#        "session_id": "results_2025_09_26__14_41_18",
#        "analysis_result": "analysis_result"
#    }
#}
#feedback_db[persona_id].update(feedback_entry)

#result_hash = 123
#feedback_entry = {
#    result_hash: {
#        "session_id": "new",
#        "analysis_result": "analysis_result"
#    }
#}
#feedback_db[persona_id].update(feedback_entry)



print(feedback_db)


{'persona_id': 'persona_001', 'predicted_type': 'awareness', 'predicted_items': 'too_young'}
-6946375721364821563
{'persona_001': {-6946375721364821563: {'session_id': 'results_2025_09_26__14_41_18', 'analysis_result': 'analysis_result'}, 123: {'session_id': 'new', 'analysis_result': 'analysis_result'}}}


In [49]:
agent_message = "Hello! I'm coming back to you to give you feed back regarding our last interview.\n"

if result['predicted_type'] == 'awareness':
    if result['predicted_type'] == 'too_young':
        agent_message += "As your age is below 16, I will only provide you awereness about topics you are interested in.\n"
        agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
    else:
        agent_message += "As your are only interested by awereness (no training nor jib), let's discuss about .\n"
        agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
elif result['predicted_type'] == 'trainings_only':
    agent_message += "I have identified that you are interested in training only. Your are not seeking for jobs.\n"
    agent_message += "Hereafter if the list of trainings I find for .\n"
    agent_message += "Please tell me if that fit for you. If not, explain me why so that I can adapt my research for you.\n"
    agent_message += "\nTRAININGS DESCRIPTION :\n"

    jobs = result['trainings']
    
    trainings = result['trainings']
    description = []
    for training in trainings:
        id = training
        filename = f"{id}.md"
        path = trainings_dir / filename
        # print(job_path)
        p = Path(path)
        content = p.read_text(encoding='utf-8', errors='ignore')
        # print(content)
        agent_message += "\n\n------ TRAINING -------\n"
        agent_message += content

    # print(agent_message)

    # resp = send_message_to_chat(agent_message, persona_id, conversation_id)
    # print(resp)

In [20]:
persona_id = result['persona_id']
resp = send_message_to_chat(agent_message, persona_id)
print(resp)

('oh hi... umm i think u got it wrong? i want trainig not just awereness. i wanna lern how food is made safe and clean. like the mashines and rules. i can lern fast if i see the food lines? maybe u find me a place to start? im 21 but still in school so i need trainig first.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_001#2025-09-26T14:49:44')


In [ ]:
selected_file = "results_2025_09_25__20_50_29.json"
results_save_path = SUBMISSION_DIR / selected_file
results = read_json(results_save_path)

persona_index = 0

result = results[persona_index]

# print(result)

persona_id = result['persona_id']
print(f"persona_id : {persona_id}")

jobs = result['jobs']
# print(f"persona_id : {jobs}")

jobs_description = []

for job in jobs:
    job_id = job['job_id']
    # print(job_id)
    filename = f"{job_id}.md"
    job_path = jobs_dir / filename
    # print(job_path)
    p = Path(job_path)
    content = p.read_text(encoding='utf-8', errors='ignore')
    # print(content)
    jobs_description.append(content)

# print(jobs_description)

In [ ]:
index = 0
conversation_id = None

agent_message = "Hello! I'm here to help you find the best opportunities.\n"
agent_message += "Hereafter is a job I can propose to you.\n"
agent_message += "Please tell me is that fit for you. If not, explain me why so that I can adapt my research for you.\n"
agent_message += "\nJOB DESCRIPTION :\n"
agent_message += jobs_description[index]

print(agent_message)

resp = send_message_to_chat(agent_message, persona_id, conversation_id)

print(resp)